## Prepare data reader and model execution context

In [6]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "account_abstraction"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="25821200:+1",
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-02-10 20:56:09 [info     ] prepared 2 input batches.      filename=byblock.py lineno=88 process=37187
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain',
                                                      value='base'),
                                      PartitionColumn(name='dt',
                                                      value='2025-02-01')]),
           read_from=DataLocation.GCS,
           dataset_paths={'blockbatch/account_abstraction_prefilter/entrypoint_logs_v1': ['gs://oplabs-tools-data-sink/blockbatch/account_abstraction_prefilter/entrypoint_logs_v1/chain=base/dt=2025-02-01/000025820800.parquet'],
                          'blockbatch/account_abstraction_prefilter/entrypoint_traces_v1': ['gs://oplabs-tools-data-sink/blockbatch/account_abstraction_prefilter/entrypoint_traces_v1/chain=base/dt=2025-02-01/000025820800.parquet']},
           inputs_ready=True,
           extra_marker_data={'max_block': 25821200,
                              '

In [7]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

register_4337_decoders(ctx)

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)


# Decoded UserOperationEvent logs.
user_ops = auxiliary_templates["account_abstraction/useroperationevent_logs"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets["blockbatch/account_abstraction_prefilter/entrypoint_logs_v1"].as_subquery(),
    },
)



2025-02-10 20:56:09 [warning  ] duckdb function is already registered: decode_useroperationevent filename=register.py lineno=24 process=37187
2025-02-10 20:56:09 [warning  ] duckdb function is already registered: decode_innerhandleop filename=register.py lineno=24 process=37187
2025-02-10 20:56:09 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=37187
2025-02-10 20:56:09 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=37187 template=account_abstraction/useroperationevent_logs
2025-02-10 20:56:10 [info     ] duck db size: 48.2MB           filename=client.py lineno=36 process=37187


In [8]:
# Traces initiated on behalf of the UserOperationEvent sender
entrypoint_traces = auxiliary_templates["account_abstraction/enriched_entrypoint_traces"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": input_datasets["blockbatch/account_abstraction_prefilter/entrypoint_traces_v1"].as_subquery(),
        "innerhandleop_method_ids": ", ".join(
            [
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0}'",
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0}'",
            ]
        ),
    },
)

2025-02-10 20:56:10 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=37187
2025-02-10 20:56:10 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=37187 template=account_abstraction/enriched_entrypoint_traces
2025-02-10 20:56:12 [info     ] duck db size: 48.2MB           filename=client.py lineno=36 process=37187


In [9]:
auxiliary_templates["account_abstraction/data_quality_check_01"].run_as_data_quality_check(ctx)

2025-02-10 20:56:12 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=37187 template=account_abstraction/data_quality_check_01


[]

In [10]:
auxiliary_templates["account_abstraction/data_quality_check_02"].run_as_data_quality_check(ctx)

2025-02-10 20:56:12 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=37187 template=account_abstraction/data_quality_check_02


[]

In [11]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────────────────────────────┐
│                      name                       │
│                     varchar                     │
├─────────────────────────────────────────────────┤
│ account_abstraction__enriched_entrypoint_traces │
│ account_abstraction__useroperationevent_logs    │
└─────────────────────────────────────────────────┘

In [12]:
ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        trace_root, 
        trace_address, 
        matched_userop_trace_address, 
        matched_userop_sender, 
        is_from_matched_userop_sender, 
        innerhandleop_opinfo_sender, 
        from_address, 
        to_address
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────┬───────────────┬──────────────────────────────┬────────────────────────────────────────────┬───────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┐
│                          transaction_hash                          │ trace_root │ trace_address │ matched_userop_trace_address │           matched_userop_sender            │ is_from_matched_userop_sender │        innerhandleop_opinfo_sender         │                from_address                │                 to_address                 │
│                              varchar                               │   int32    │    varchar    │           varchar            │                  varchar                   │            boolean            │                  varchar                   │                  varchar                   │                 

In [49]:
ctx.client.sql(f"""
    SELECT 
        *
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    AND trace_address = '5,0,0'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(render_mode=1, max_width=360, max_col_width=32)

┌─────────────────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│             Column              │  Type   │                                                                                                                               Row 1                                                                                                                                │
├─────────────────────────────────┼─────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ dt                              │ date    │                                  

In [48]:
ctx.client.sql(f"""
    SELECT 
        *
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    AND trace_address = '6'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(render_mode=1, max_width=360, max_col_width=32)

┌─────────────────────────────────┬─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│             Column              │  Type   │                                                                                                                                                          Row 1                                                                                                                                                           │
├─────────────────────────────────┼─────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [34]:
import polars as pl

rel = ctx.client.sql(f"""
    SELECT * FROM {user_ops} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    ORDER BY log_index
    """)

with pl.Config(
    tbl_cols=-1,
    tbl_rows=-1,
    tbl_width_chars=1200,
    fmt_str_lengths=128,
    tbl_formatting="MARKDOWN",
    tbl_hide_column_data_types=False,
    tbl_hide_dataframe_shape=True,
) as cfg:
    print(rel.pl())


rel


| dt         | chain | chain_id | network | block_timestamp | block_number | block_hash                                                         | transaction_hash                                                   | transaction_index | log_index | contract_address                           | userophash                                                         | sender                                     | paymaster                                  | decoded_json                                                                           |
| ---        | ---   | ---      | ---     | ---             | ---          | ---                                                                | ---                                                                | ---               | ---       | ---                                        | ---                                                                | ---                                        | ---                                        | ---      

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │                             userophash                             │                   sender                   │                 paymaster                  │      

In [37]:
with pl.Config(
    tbl_cols=-1,
    tbl_rows=-1,
    tbl_width_chars=1200,
    fmt_str_lengths=128,
    tbl_formatting="MARKDOWN",
    tbl_hide_column_data_types=True,
    tbl_hide_dataframe_shape=True,
) as cfg:
    df1 = ctx.client.sql(f"DESCRIBE TABLE {user_ops}").select("column_name", "column_type").pl()
    print(df1)
    print()
    
    df2 = ctx.client.sql(f"DESCRIBE TABLE {entrypoint_traces}").select("column_name", "column_type").pl()
    print(df2)
    print()

| column_name       | column_type |
|-------------------|-------------|
| dt                | DATE        |
| chain             | VARCHAR     |
| chain_id          | INTEGER     |
| network           | VARCHAR     |
| block_timestamp   | UINTEGER    |
| block_number      | BIGINT      |
| block_hash        | VARCHAR     |
| transaction_hash  | VARCHAR     |
| transaction_index | BIGINT      |
| log_index         | BIGINT      |
| contract_address  | VARCHAR     |
| userophash        | VARCHAR     |
| sender            | VARCHAR     |
| paymaster         | VARCHAR     |
| decoded_json      | VARCHAR     |

| column_name                     | column_type |
|---------------------------------|-------------|
| dt                              | DATE        |
| chain                           | VARCHAR     |
| chain_id                        | INTEGER     |
| network                         | VARCHAR     |
| block_timestamp                 | UINTEGER    |
| block_number                    | B